In [ ]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

In [30]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [31]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [32]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [33]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Combine last 2 steps, and zoom figure
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Heatmap of windspeed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [39]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kuala Terengganu,MY,2021-07-18 03:36:20,5.3302,103.1408,85.95,84,20,3.44
5,5,Nishihara,JP,2021-07-18 03:36:10,26.1842,127.7558,89.26,80,75,13.80
6,6,Georgetown,MY,2021-07-18 03:33:53,5.4112,100.3354,88.90,73,20,1.01
10,10,Kapaa,US,2021-07-18 03:33:50,22.0752,-159.3190,85.98,79,75,5.01
19,19,Atlantic City,US,2021-07-18 03:33:44,39.3643,-74.4229,78.98,81,40,4.61
21,21,Lianzhou,CN,2021-07-18 03:36:26,24.7811,112.3825,85.68,79,100,2.68
22,22,Miri,MY,2021-07-18 03:34:21,4.4148,114.0089,87.94,79,40,6.91
25,25,Tuktoyaktuk,CA,2021-07-18 03:34:32,69.4541,-133.0374,78.80,41,40,9.22
33,33,Kidal,ML,2021-07-18 03:36:31,18.4411,1.4078,88.52,41,6,9.04
34,34,Atuona,PF,2021-07-18 03:34:46,-9.8000,-139.0333,77.74,80,22,26.69


In [40]:
# Check for nulls
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       1
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [41]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kuala Terengganu,MY,85.95,5.3302,103.1408,
5,Nishihara,JP,89.26,26.1842,127.7558,
6,Georgetown,MY,88.90,5.4112,100.3354,
10,Kapaa,US,85.98,22.0752,-159.3190,
19,Atlantic City,US,78.98,39.3643,-74.4229,
21,Lianzhou,CN,85.68,24.7811,112.3825,
22,Miri,MY,87.94,4.4148,114.0089,
25,Tuktoyaktuk,CA,78.80,69.4541,-133.0374,
33,Kidal,ML,88.52,18.4411,1.4078,
34,Atuona,PF,77.74,-9.8000,-139.0333,


In [42]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [52]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
hotel_df.head(20)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kuala Terengganu,MY,85.95,5.3302,103.1408,Hotel Grand Continental Kuala Terengganu (4 Star)
5,Nishihara,JP,89.26,26.1842,127.7558,Minshuku Agaihama
6,Georgetown,MY,88.90,5.4112,100.3354,Cititel Penang
10,Kapaa,US,85.98,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Atlantic City,US,78.98,39.3643,-74.4229,Courtyard by Marriott Atlantic City Beach Block
21,Lianzhou,CN,85.68,24.7811,112.3825,Qingcheng Tourism Industry
22,Miri,MY,87.94,4.4148,114.0089,"Mega Hotel, Miri"
25,Tuktoyaktuk,CA,78.80,69.4541,-133.0374,End of The Road Ltd.
33,Kidal,ML,88.52,18.4411,1.4078,
34,Atuona,PF,77.74,-9.8000,-139.0333,Villa Enata


In [55]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [56]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))